How to detect a human name?

Maybe I can do that with scikit-lean?

First I need a list of names, to train a model for detecting names. I have some
names, that I have [scraped some time
ago](https://github.com/sirex/databot-bots/blob/master/bots/tevu-darzelis.lt/vaiku-vardai.py),
I will try to use that.

In [31]:
import os
import sqlite3
import msgpack

In [32]:
conn = sqlite3.connect(os.path.expanduser ('~/devel/databot/bots/data/tevu-darzelis.lt/vaiku-vardai.db'))
curs = conn.cursor()
curs.execute('SELECT id FROM databotpipes WHERE pipe = ?', ('vardai',))
tid, = curs.fetchone()
names = [
    msgpack.unpackb(row[0], strict_map_key=False)[1]['vardas'].lower()
    for row in curs.execute(f'SELECT value FROM t{tid}')
]

In [33]:
len(names)

13321

In [25]:
names[:3]

['audrius', 'atėnė', 'alanas']

In [53]:
def ngram(x, n):
    return list(zip(*[x[i:] for i in range(n)]))

def build(X):
    model = {}
    for x in X:
        for i, k in enumerate(ngram(x, 2)):
            model.setdefault(i, {})
            model[i].setdefault(k, 0)
            model[i][k] += 1
    return {
        i: {k: v / max(x.values()) for k, v in x.items()}
        for i, x in model.items()
    }

model = build(names)

In [59]:
def check(model, x):
    p = [
        model.get(i, {}).get(k, 0)
        for i, k in enumerate(ngram(x, 2))
    ]
    return sum(p) / len(p)

check(model, 'kamuolys')

0.14921365705997422

In [62]:
!python -m spacy download lt_core_news_md

     |████████████████████████████████| 44.9 MB 1.7 MB/s eta 0:00:01
Using legacy 'setup.py install' for lt-core-news-md, since package 'wheel' is not installed.
    Running setup.py install for lt-core-news-md ... - \ done
You should consider upgrading via the '/home/sirex/devel/sirex.lt/nikola/env/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('lt_core_news_md')


In [12]:
import spacy

nlp = spacy.load('lt_core_news_md')

In [13]:
doc = nlp('Bandymas')

In [14]:
list(doc)

[Bandymas]

In [29]:
nlp('Mantas Zimnickas').ents[0].label_

'PERSON'

In [35]:
num_person = sum(sum(e.label_ == 'PERSON' for e in nlp(n).ents) for n in names)

In [36]:
num_person / len(names) * 100


0.4804444110802492